## import packages

In [1]:
import numpy as np
import pandas as pd
from typing import Optional, Tuple
import sys

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

from pickle import dump
from sklearn.preprocessing import MinMaxScaler
import time
from tqdm import tqdm

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("cuda is available")
else:
    print("cuda is NOT available")


import shutil
import warnings
import pickle

warnings.filterwarnings("ignore")
import logging

logging.disable(logging.CRITICAL)

from nn_functions import surrogate
from moving_average import moving_average_1d
import copy
from GAMMA_obj_temp_depth import GAMMA_obj

import sys
sys.path.append('../1_model')
from TiDE import TideModule, quantile_loss, TiDE_forward


cuda is available


## read data

In [6]:
df_print = pd.read_csv('merged_df_2_99_temp_depth.csv')
output_dir = 'split_by_laser_power_number'
os.makedirs(output_dir, exist_ok=True)

unique_laser_numbers = sorted(df_print["laser_power_number"].unique())
for laser_num in unique_laser_numbers:
    df_filtered = df_print[df_print["laser_power_number"] == laser_num]
    output_path = os.path.join(output_dir, f"laser_power_number_{int(laser_num)}.csv")
    df_filtered.to_csv(output_path, index=False)
print(f"Saved: {output_path} ({len(df_filtered)} rows)")

Saved: split_by_laser_power_number/laser_power_number_99.csv (6295 rows)


In [8]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os

# Base directory containing the laser_profile_*.csv files
base_dir = "/home/ftk3187/github/DPC_research/02_DED/2_policy/data/laser_inputs/single_track_square/laser_power_profiles/csv"

# Loop through laser_profile_2.csv to laser_profile_99.csv
for laser_num in range(2, 100):
    filename = f"laser_profile_{laser_num}.csv"
    path = os.path.join(base_dir, filename)

    # Skip if the file does not exist
    if not os.path.exists(path):
        print(f"⚠ File not found: {filename}")
        continue

    # Read the CSV file
    df = pd.read_csv(path)
    original_len = len(df)
    target_len = original_len * 30  # 30x upsampling

    # Create normalized x-axis for interpolation
    x_original = np.linspace(0, 1, original_len)
    x_target = np.linspace(0, 1, target_len)

    # Define interpolation functions
    interp_power = interp1d(x_original, df['laser_power'], kind='linear')
    interp_time = interp1d(x_original, df['time_elapsed'], kind='linear')

    # Generate interpolated values
    laser_power_interp = interp_power(x_target)
    time_elapsed_interp = interp_time(x_target) * 5  # scale time

    # Create new DataFrame with interpolated values
    df_interp = pd.DataFrame({
        'laser_power': laser_power_interp,
        'time_elapsed': time_elapsed_interp
    })

    # Overwrite the original file
    df_interp.to_csv(path, index=False)
    print(f"✔ Updated and saved: {filename} ({target_len} rows)")


✔ Updated and saved: laser_profile_2.csv (188850 rows)
✔ Updated and saved: laser_profile_3.csv (188850 rows)
✔ Updated and saved: laser_profile_4.csv (188850 rows)
✔ Updated and saved: laser_profile_5.csv (188850 rows)
✔ Updated and saved: laser_profile_6.csv (188850 rows)
✔ Updated and saved: laser_profile_7.csv (188850 rows)
✔ Updated and saved: laser_profile_8.csv (188850 rows)
✔ Updated and saved: laser_profile_9.csv (188850 rows)
✔ Updated and saved: laser_profile_10.csv (188850 rows)
✔ Updated and saved: laser_profile_11.csv (188850 rows)
✔ Updated and saved: laser_profile_12.csv (188850 rows)
✔ Updated and saved: laser_profile_13.csv (188850 rows)
✔ Updated and saved: laser_profile_14.csv (188850 rows)
✔ Updated and saved: laser_profile_15.csv (188850 rows)
✔ Updated and saved: laser_profile_16.csv (188850 rows)
✔ Updated and saved: laser_profile_17.csv (188850 rows)
✔ Updated and saved: laser_profile_18.csv (188850 rows)
✔ Updated and saved: laser_profile_19.csv (188850 rows)
